# Funkcionalno programiranje

Amer Hasanović

## Organizacija koda

`F#` kod se zapisuje u tekstualne fajlove sa sljedećim ekstenzijama:

* `fs` - kod koji će se kompajlirati u `F#` projektima,
* `fsx` - skripte koje će se izvršavati bez prethodnog kompajliranja
  * mogu koristiti kod iz drugih fajlova pomoću direktive:

      ```fsharp
      #load "imefajla.fs"
      ```

Skripte mogu koristiti kod iz drugih fajlova pomoću direktive `load`:

```fsharp
#load "imefajla.fs"
```

Skripte se mogu istovremeno kompajlirati i pokretati u formi:

```text
dotnet fsi ime_skripte.fsx
```

* Ukoliko se u gornjoj komandi izostavi ime skripte, pokreće se interaktivno okruženje za unos `F#` koda, tzv **repl**.

`F#` kod zapisan u fajlovima koji se bavi istim ili sličnim konceptima, radi smanjivanja vjerovatnoće konflikata u imenima, grupiše se u kohezivne cjeline i to:

* `namespace`
* `module`

## `namespace`

Predstavlja kontejner za kod koji može da sadrži:
* `namespace`,
* `module`, 
*  tipove


Namespace se može kreirate i/ili otvoriti u sljedećem formatu:

```fsharp
namespace ns_ime
```

* Sav kod unutar jednog fajla, od gornje deklaracije, do nove `namespace` deklaracije, pripada `ns_ime` namespace-u.



Namespace je otvoren, što znači da svaki fajl može dodavati novi sadržaj u namespace ponovnim korištenjem `namespace` deklaracije.

* Prvo otvaranje namespace-a predstavlja njegovo kreiranje.

Kod unutar `namespace` deklaracije se ne indentira.

Namespace može biti ugnježden u drugi namespace, pri čemu se imenuje na način:

```fsharp
namespace parent_ns.child_ns
```

* `child_ns` - je ime ugnježdenog namespace-a

Simboli iz nekog namespace-a mogu se koristiti u drugom namespace-u korištenjem kvalifikovanog imena simbola koje uključuju i ime namespace-a u kojem je simbol definiran.

* U kvalifikovanom imenu između imena namespace kojem simbol pripada i imena simbola stavlja se `.`

Simboli iz jednog namespace-a mogu se referencirati direktno u drugom namespace-u uz prethodnu deklaracije `open` u formatu:

```fsharp
open ns1
```

* `ns1` je ime namespace čiji simboli će se referencirati bez potpune kvalifikacije u trenutnom namespace-u.

U slučaju da dođe do konflikta prilikom nekvalifikovane upotrebe  simbola koji je definiran u dva namespace-a koji su otvoreni sa `open` deklaracijom, kompajler bira zadnje viđeno ime.

```fsharp
// fajl Foo.fs
namespace Foo
type Tar = A | B

namespace Foo.Bar
type Foobar = C | D

namespace Test
type Tar = { name : string; age : int }



```fsharp
// fajl script1.fsx
#load "Foo.fs"
let a : Foo.Tar = Foo.A 
let t : Foo.Bar.Foobar = Foo.Bar.C
let k : Test.Tar = { name = "foo"; age = 23 }

```fsharp
// fajl script2.fsx
#load "Foo.fs"

open Test
open Foo

let t : Tar = A
let b : Test.Tar = { name = "foo"; age = 80 }




## `module`

Predstavlja kontejner za kod koji može da sadrži:

* `module`,
* tipove,
* varijable

Modul može biti:

* top level 
  * može se definirati samo na početku fajla i sadrži cijeli fajl
  * kod koji pripada modulu se ne indentira.
* local
  * kod koji pripada modulu se indentira.

Top level modul deklaracija:

```fsharp
// početak fajla
module Foobar
// sadržaj modula
```

Local modul deklaracija:

```fsharp
// bilo gdje u fajlu
module Foobar = 
  // sadržaj modula
```

Top level modul se prilikom deklaracije može ubaciti u namespace u sljedećem formatu:

```fsharp
// početak fajla
module ns_ime.module_ime
// sadržaj modula
```

Ukoliko se `F#` projekat sastoji od jednog fajla, a njegov sadržaj nije eksplicitno postavljen unutar nekog modula i namespace-a, kompajler će implicitno na osnovu imena fajla kreirati top level modul. 

```fsharp
// fajl foobar.fs
let foo = 5
```

isto kao:

```fsharp
// fajl foobar.fs
module Foobar
let foo = 5
```

Ukoliko se `F#` projekat sastoji od više fajlova, na početku svakog fajla mora biti ili deklaracija namespace-a ili top level module-a.


Slično kao za namespace, simboli iz modula mogu se koristiti u drugim modulima kroz kvalificiran pristup,  ili direktno, uz prethodnu upotrebu `open` deklaracije.

Za razliku od namespace-a, moduli mogu imati kod samo u jednom fajlu.


Modul može biti automatski otvoren kada se otvori modul ili namespace u kojim je ugnježden, ukoliko se prilikom deklaracije modula koristi atribut `AutoOpen`.

```fsharp
module TopL

[<AutoOpen>]
module Local1 = 
  // sadržaj
```

```fsharp
// fajl ex.fs
namespace Example
type PersonType = {First:string; Last:string}

module Person =
    let create first last =
        {First=first; Last=last}

    let fullName {First=first; Last=last} =
        first + " " + last

```

```fsharp
// fajl prog.fsx
#load "ex.fs"

let foo 
    = Example.Person.create "foo" "bar"
```

## Upravljanje projektima

`F#` fajlovi se grupišu u projekte različite namjene, npr:

* terminal aplikacija, tzv console projekat
* biblioteka, tzv classlib projekat


Kompajliranjem i uvezivanjem fajlova koji pripadaju projektu nastaju  **Assemblies**:

* fajlovi koje može koristiti `dotNet` virtuelna mašina za izvršavanje `dotNet` programa, (sadrže bytecode)
* imaju ekstenziju `dll`

Projekti se mogu radi lakšeg upravljanja i kompajliranja grupisati u koncept `Solution`

```bash
dotnet new classlib --language F# -n lib1
dotnet new classlib --language F# -n lib2
dotnet new console --language F# -n app
dotnet add app/app.fsproj reference lib1/lib1.fsproj
dotnet add app/app.fsproj reference lib2/lib2.fsproj
dotnet new sln -n coolapp
dotnet sln add app/app.fsproj
dotnet sln add lib1/lib1.fsproj
dotnet sln add lib2/lib2.fsproj
```